In [57]:
import pandas as pd
import sklearn.linear_model as sk
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
import nltk
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import pickle
from string import punctuation
from sklearn.metrics import accuracy_score
import re

In [58]:
#nltk.download("all") 

In [59]:
tweets=pd.read_csv("Datasets/NoThemeTweets.csv")

In [60]:
tweets.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:)
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:)
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:)
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:)
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:)


In [61]:
sentimento_tweets=tweets["sentiment"].replace(["Negativo","Positivo"],[0,1])

In [62]:
tweets["sentimento"]=sentimento_tweets

In [63]:
tweets["text_pt"]=tweets["tweet_text"]
tweets = tweets[['text_pt', 'sentimento']]

In [64]:
tweets["sentimento"].value_counts()

sentimento
0    522707
1    263107
Name: count, dtype: int64

In [65]:
tweets.head()

,text_pt,sentimento
0,@Tixaa23 14 para eu ir :),1
1,@drexalvarez O meu like eu já dei na época :),1
2,Eu só queria conseguir comer alguma coisa pra ...,1
3,:D que lindo dia !,1
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",1


In [66]:
resenhas=pd.read_csv("Datasets/imdb-reviews-pt-br.csv")

In [67]:
resenhas.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [68]:
sentimento_resenhas=resenhas["sentiment"].replace(["neg","pos"],[0,1])

In [69]:
resenhas["sentimento"]=sentimento_resenhas

In [70]:
resenhas= resenhas[['text_pt', 'sentimento']]
resenhas.head()

,text_pt,sentimento
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",0
1,Este é um exemplo do motivo pelo qual a maiori...,0
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",0
3,Nem mesmo os Beatles puderam escrever músicas ...,0
4,Filmes de fotos de latão não é uma palavra apr...,0


In [71]:
df=resenhas
#df = pd.concat([tweets, resenhas], ignore_index=True)
sentiment=df["sentimento"].replace([0,1],["Negativo","Positivo"])

In [72]:
df["sentiment"]=sentiment

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49459 entries, 0 to 49458
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text_pt     49459 non-null  object
 1   sentimento  49459 non-null  int64 
 2   sentiment   49459 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [74]:
df.head()

,text_pt,sentimento,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",0,Negativo
1,Este é um exemplo do motivo pelo qual a maiori...,0,Negativo
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",0,Negativo
3,Nem mesmo os Beatles puderam escrever músicas ...,0,Negativo
4,Filmes de fotos de latão não é uma palavra apr...,0,Negativo


In [75]:
df=df.dropna()

In [76]:
todos_palavras = ' '.join([texto for texto in df["text_pt"]])

In [77]:
token_espaco = tokenize.WhitespaceTokenizer()
todas_palavras = ' '.join([texto for texto in df["text_pt"]])

In [78]:
def remove_emoticons(text):
    emoticon_pattern = r'[:;=][\-\^]?[\)\(DPp@#$&|]'
    return re.sub(emoticon_pattern, '', text)

In [79]:
palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
frase_processada = list()

for opiniao in df["text_pt"]:
    nova_frase = list()
    palavras_texto = token_espaco.tokenize(remove_emoticons(opiniao))
    for palavra in palavras_texto:
        if palavra not in palavras_irrelevantes:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
df["tratamento_1"] = frase_processada

In [80]:
token_pontuacao = tokenize.WordPunctTokenizer()

In [81]:
pontuacao = list()
for ponto in punctuation:
    pontuacao.append(ponto)

pontuacao_stopwords = pontuacao + palavras_irrelevantes

frase_processada = list()
for opiniao in df["tratamento_1"]:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
df["tratamento_2"] = frase_processada

In [82]:
import unidecode
sem_acentos=[unidecode.unidecode(texto) for texto in df["tratamento_2"]]

In [83]:
df["tratamento_3"]=sem_acentos
frase_processada = list()
for opiniao in df["tratamento_3"]:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
df["tratamento_3"]=frase_processada

In [84]:
stopwords_sem_acento=[unidecode.unidecode(texto) for texto in pontuacao_stopwords]

In [85]:
frase_processada = list()
for opiniao in df["tratamento_3"]:
    nova_frase = list()
    opiniao=opiniao.lower()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in stopwords_sem_acento:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
df["tratamento_4"]=frase_processada

In [86]:
stemmer=nltk.RSLPStemmer()

In [87]:
frase_processada = list()
for opiniao in df["tratamento_4"]:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        nova_frase.append(stemmer.stem(palavra))
    frase_processada.append(' '.join(nova_frase))
    
df["tratamento_5"]=frase_processada

In [88]:
df.head()

,text_pt,sentimento,sentiment,tratamento_1,tratamento_2,tratamento_3,tratamento_4,tratamento_5
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",0,Negativo,"Mais vez, Sr. Costner arrumou filme tempo nece...",Mais vez Sr Costner arrumou filme tempo necess...,Mais vez Sr Costner arrumou filme tempo necess...,vez sr costner arrumou filme tempo necessario ...,vez sr costn arrum film temp necessari alem te...
1,Este é um exemplo do motivo pelo qual a maiori...,0,Negativo,Este exemplo motivo maioria filmes ação mesmos...,Este exemplo motivo maioria filmes ação mesmos...,Este exemplo motivo maioria filmes acao mesmos...,exemplo motivo maioria filmes acao mesmos gene...,exempl motiv maior film aca mesm gener chat na...
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",0,Negativo,"Primeiro tudo odeio raps imbecis, poderiam agi...",Primeiro tudo odeio raps imbecis poderiam agir...,Primeiro tudo odeio raps imbecis poderiam agir...,primeiro tudo odeio raps imbecis poderiam agir...,prim tud odei rap imbecil pod agir arm pressio...
3,Nem mesmo os Beatles puderam escrever músicas ...,0,Negativo,Nem Beatles puderam escrever músicas todos gos...,Nem Beatles puderam escrever músicas todos gos...,Nem Beatles puderam escrever musicas todos gos...,beatles puderam escrever musicas todos gostass...,beatl pud escrev music tod gost emb walt hill ...
4,Filmes de fotos de latão não é uma palavra apr...,0,Negativo,"Filmes fotos latão palavra apropriada eles, ve...",Filmes fotos latão palavra apropriada verdade ...,Filmes fotos latao palavra apropriada verdade ...,filmes fotos latao palavra apropriada verdade ...,film fot lata palavr apropri verdad tant ous q...


In [89]:
x = df["tratamento_5"].values
y = df["sentimento"].values

In [90]:
# Dividindo os dados em conjuntos de treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [91]:
# Vetorizando o texto
vectorizer = TfidfVectorizer()
x_train_vect = vectorizer.fit_transform(x_train)
x_test_vect = vectorizer.transform(x_test)

In [92]:
# Treinando o modelo
model = MultinomialNB()
model.fit(x_train_vect, y_train)

MultinomialNB()

In [93]:
# Fazendo previsões e avaliando o modelo
y_pred = model.predict(x_test_vect)
print("Acurácia:", accuracy_score(y_test, y_pred))

Acurácia: 0.8539223615042458


In [94]:
import pickle

pickle.dump(model, open("lrModel", "wb"))
pickle.dump(vectorizer, open("vectorizer", "wb"))

In [95]:
model = pickle.load(open("lrModel", "rb"))
vectorizer = pickle.load(open("vectorizer", "rb"))

In [96]:
from tratamento import trata

In [97]:
def classifica_tweet(text):
    text = vectorizer.transform(trata(text))
    return model.predict(text)

In [98]:
text=['como eu amo esses caras']

In [99]:
prediction = classifica_tweet(text)

print(prediction)

[1]


In [100]:
text=['como eu odeio esses caras']

In [101]:
prediction = classifica_tweet(text)

print(prediction)

[0]
